###### Webscrape_github
This is a cleaned up version of my webscrape code that begins with a set
or urls of startup companies typically downloaded from Crunchbase
with a given search pattern. The
current input file (included in this repo) is Printingtot_v4.csv.

The code then scrapes the text from the front page
(the Scrapy package is used):
>sitep=response.xpath('//p/text()').getall()

The spider MySpider first modifies the input url list (which is
accompanied by other data from Crunchbase including
>'Organization Name','Headquarters Location','Full Description','Website')

and removes non-US companies and cleans up for nan records.

The parse function takes the response and pulls off the text
and the urls and also replaces unreadable characters (possibly
this would not be necessary with proper specification of 
file code as utf-8).

The principal output is a file that includes, for each record, the
scraped url, the company name and the text. The current name is
Ptotal_v2.cor.

Currently, searches (such as "inkjet") when successful go to basic pipeline
and get stuff printed out in a separate file.

In [1]:
import scrapy
import logging
import json
from scrapy.crawler import CrawlerProcess
import pandas as pd
from pandas import read_csv
import csv
from scrapy.shell import inspect_response
import numpy as np
from scrapy.pipelines.images import ImagesPipeline
from scrapy.selector import Selector
import difflib

In [2]:
class BasicPipeline(object):
    
    def open_spider(self,spider):
        self.file = open('Ptotal_v2.txt','w')
        
    def close_spider(self,spider):
        self.file.close()
        
    def process_item(self,item,spider):
        line=str(item)
        self.file.write(line + "\n")
        return item

In [3]:
class MyNewPipeline(object):
    
    def open_spider(self,spider):
        self.file = open('items2.jl', 'wb')
        
    def close_spider(self, spider):
        self.file.close()
        
    def process_item(self,item,spider):
        print('processing',dict(item))
        line = json.dumps(dict(item)) + " new " + "\n"
        self.file.write(line)
        return item

In [4]:
class ItemCollectorPipeline(object):
    def __init__(self):
        self.ids_seen = set()
        
    def process_item(self,item,spider):
        print("ItemCollector")
        items.append(item)

In [5]:
class ImageItem(scrapy.Item):
    images = scrapy.Field()
    image_urls = scrapy.Field()

In [6]:
class ItemTest(scrapy.Item):
    Organization = scrapy.Field()
    url = scrapy.Field()
    robot = scrapy.Field()
    machine = scrapy.Field()
    tinyML = scrapy.Field()
    brain = scrapy.Field()
    food = scrapy.Field()
    textout = scrapy.Field()

In [7]:
class Print_Item(scrapy.Item):
    Organization = scrapy.Field()
    url = scrapy.Field()
    ThreeDP = scrapy.Field()
    inkjet = scrapy.Field()
    digitalP = scrapy.Field()
    biomaterials = scrapy.Field()
    food = scrapy.Field()
    ScreenPrinting = scrapy.Field()
    textout = scrapy.Field()
    searchterm = scrapy.Field()
    fulltext = scrapy.Field()

In [8]:
class Scrapy2Item(scrapy.Item):
    title = scrapy.Field()
    image_urls = scrapy.Field()
    sku = scrapy.Field()

class Scrapy2Pipeline(ImagesPipeline):
    def get_media_requests(self, item, info):
        return [scrapy.Request(x, meta={'image_name': item['sku']})
                for x in item.get('image_urls', [])]

    def file_path(self, request, response=None, info=None):
        return '%s.jpg' % request.meta['image_name']

In [9]:
class MySpider(scrapy.Spider):
    name = "startups"
    
# read crunchbase file and pick off websites
    dataset1=read_csv('Printingtot_v4.csv')
    df=dataset1[['Organization Name','Headquarters Location','Website']]
    dfT=dataset1[['Full Description','Website']]
    dfn=df.to_numpy()
    dfTn=dfT.to_numpy()
    dfnl=dfn.tolist()
    dfTnl=dfTn.tolist()
    print('df.shape',df.shape,dfnl[3][1])
    print('dfT.shape',dfT.shape)
    ii1=-1
    ii2=-1
    ii3=-1
    df2=[]
    dfT2=[]
    for kk in range(dfn.shape[0]): # eliminate nans, specify U.S.
        if str(dfn[kk,2]) != 'nan' and str(dfn[kk,1]) != 'nan': # use numpy for tests
            if 'United States' in dfn[kk,1]:
                ii1+=1
                df2.append(dfnl[kk][:]) # use lists for appending (not easy with numpy)
                dfT2.append(dfTnl[kk][:])
            else:
                ii2+=1
        else:
            ii3+=1


#    print('ii1',ii1,'ii2',ii2,'ii3',ii3)
    df2n=np.array(df2) # convert list to numpy - easier to slice
    dfT2n=np.array(dfT2)
    print('df2n.shape',df2n.shape,'dfT2n.shape',dfT2n.shape)
    minc=0
    maxc=df2n.shape[0]
    fulltextCB=dfT2n[minc:maxc,0]
    companies=df2n[minc:maxc,0].tolist() # slice numpy array, change it to a list
    start_urlsB=df2n[minc:maxc,2] # numpy - this is an intermediate step, start_urlsB no longer used
    start_urls=start_urlsB.tolist() # basic starting urls for scrapy - limited to U.S., nans eliminated
    iout1=-1
    iout2=-1

    custom_settings = {
        'LOG_LEVEL': logging.DEBUG,
        'ITEM_PIPELINES':{'__main__.ItemCollectorPipeline':10},
        'ITEM_PIPELINES':{'scrapy.pipelines.images.ImagesPipeline':100},
        'ITEM_PIPELINES':{'MyNewPipeline':6},
        'ITEM_PIPELINES':{'__main__.BasicPipeline':4},
    }
    
    count=0
    url_list=[]
    def parse(self, response):
        search1='/technology'
        search2='careers'
        search_list1=['inkjet head','digital printing','ink materials','additive manufacturing',\
                     'direct digital printing','biomaterials','printing for food','screen printing',\
                    'robotic arm','digital manufacturing','inkjet','dimatix']
        search_list2=['webscraping']
        search_list=search_list1
        sitep=response.xpath('//p/text()').getall()
        siteHref=response.xpath('//a/@href').getall()
        badchars = ["\U0001f48c","\U0001f4f1","\u2011","\u301d","\ufb01","\u2023","\u011f","\u1ea3","\U0001f973",\
                    "\U0001f973","\U0001f4da","\U0001f33f","\u2192","\u200d","\u2028","\u24d2","\u2011",\
                   "\u010d","\U0001f64c","\u200b","\ufb02","\ufeff","\u202f","u2070","\u2009",\
                    "\u03bc","\u27a1","\u2033","\U0001f600","\n","\t","\r"]
        bcnum=len(badchars)

        for idx,string1 in enumerate(sitep):
            for jj in range(bcnum):
                string1 = string1.replace(badchars[jj],'')

            string1 = string1.replace(u'xa0',u' ')

            if string1:
                if string1[-1] != '.':
                    string1 += '.'
                    
            sitep[idx]=string1
            
        for idy, string2 in enumerate(siteHref):
            first_hit=False
            if search1 == string2 and not(first_hit): # is 'technology' in any of the scraped links?
                first_hit=True
                technology_page=response.urljoin(string2) # cf https://docs.scrapy.org/en/latest/intro/tutorial.html
                yield scrapy.Request(technology_page, callback=self.parse)
                
                    
        out1=' '.join(sitep)

        site_url=response.url
        site_url_close=difflib.get_close_matches(site_url,self.start_urls,cutoff=0.2) # http -> https e.g.
        print('site_url',site_url,'site_url_close',site_url_close)

        if site_url_close:
            print('site_url_close[0]',site_url_close[0])
            site_urlB=site_url_close[0]
        else:
             site_urlB=site_url

        if site_urlB in self.start_urls:
            ii = self.start_urls.index(site_urlB)
        else:
            ii=-1

        print('ii',ii,'self.companies[ii]',self.companies[ii])
            
        items=Print_Item()
        
        for idz,search_term in enumerate(search_list):
            result1 = [elem for elem in sitep if search_term in elem]
            if result1:
                self.url_list.append(response.url)
                items['searchterm']=search_term
                items['Organization']=self.companies[ii]
                items['fulltext']=self.fulltextCB[ii]
                items['url']=response.url
                print('yielding item',search_term)
                yield items
                
        if out1:
            self.iout1+=1
            print('out1 true iout1',self.iout1,'url',site_url)
            file1=open("Ptotal_v2.cor",'a')
            out1=out1.lstrip(' ')
            if ii != -1: # is there a close match to this url in start_urls?
                out2=site_url + ' ' + self.companies[ii] + ' ' + out1
            else:
                out2=site_url + out1 # the url for this presumably sub-page cannot be identified 

            file1.write(out2 + '\n')
        else:
            self.iout2+=1
            print('out1 false iout2',self.iout2,'url',site_url)

        print('companies[0]',self.companies[self.count],'this url',response.url)
        self.count+=1


df.shape (14486, 3) Beijing, Beijing, China
dfT.shape (14486, 2)
df2n.shape (4854, 3) dfT2n.shape (4854, 2)


In [ ]:
outVar='outside_class'
IMAGES_STORE='C://MStopa/WebScraping/images'
items=[]
process = CrawlerProcess({
    'ITEM_PIPELINES':{'MyNewPipeline':2},
    "DOWNLOAD_DELAY":8,
    "DOWNLOAD_MAXSIZE":0
})


process.crawl(MySpider)
# process.crawl(ImgSpider)
process.start() # the script will block here until the crawling is finished

for item in items:
    print('item',item)

2021-11-30 16:35:13 [scrapy.utils.log] INFO: Scrapy 2.2.0 started (bot: scrapybot)
2021-11-30 16:35:13 [scrapy.utils.log] INFO: Versions: lxml 4.5.2.0, libxml2 2.9.10, cssselect 1.1.0, parsel 1.6.0, w3lib 1.22.0, Twisted 20.3.0, Python 3.6.10 |Anaconda, Inc.| (default, May  7 2020, 19:46:08) [MSC v.1916 64 bit (AMD64)], pyOpenSSL 19.1.0 (OpenSSL 1.1.1l  24 Aug 2021), cryptography 2.9.2, Platform Windows-10-10.0.19041-SP0
2021-11-30 16:35:13 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.selectreactor.SelectReactor
2021-11-30 16:35:13 [scrapy.crawler] INFO: Overridden settings:
{'DOWNLOAD_DELAY': 8, 'DOWNLOAD_MAXSIZE': 0, 'LOG_LEVEL': 10}
2021-11-30 16:35:14 [scrapy.extensions.telnet] INFO: Telnet Password: 9e55cfd9e8fb09e1
2021-11-30 16:35:14 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.logstats.LogStats']
2021-11-30 16:35:16 [scrapy.middleware] INFO: Enabled downloader 

site_url http://www.evolveadditive.com/ site_url_close ['http://www.evolveadditive.com/', 'http://www.cosineadditive.com/', 'https://www.lvadhesive.com/']
site_url_close[0] http://www.evolveadditive.com/
ii 3 self.companies[ii] Evolve Additive Solutions
out1 true iout1 0 url http://www.evolveadditive.com/
companies[0] VitroLabs this url http://www.evolveadditive.com/


2021-11-30 16:35:18 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.azul3d.com/> (referer: None)
2021-11-30 16:35:18 [scrapy.core.engine] DEBUG: Crawled (404) <GET https://www.digitalalloys.com/> (referer: None)
2021-11-30 16:35:18 [scrapy.downloadermiddlewares.redirect] DEBUG: Redirecting (301) to <GET https://www.sterblue.com/> from <GET http://www.sterblue.com/>
2021-11-30 16:35:18 [scrapy.dupefilters] DEBUG: Filtered duplicate request: <GET https://inkbit3d.com/technology> - no more duplicates will be shown (see DUPEFILTER_DEBUG to show all duplicates)


site_url https://www.printforia.com site_url_close ['https://www.printforia.com', 'https://www.printfp.com', 'https://www.printful.com']
site_url_close[0] https://www.printforia.com
ii 4 self.companies[ii] Printforia
out1 true iout1 1 url https://www.printforia.com
companies[0] Essentium this url https://www.printforia.com


2021-11-30 16:35:18 [scrapy.downloadermiddlewares.redirect] DEBUG: Redirecting (301) to <GET https://digifabster.com:443/> from <GET http://digifabster.com>
2021-11-30 16:35:18 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.pacificlightandhologram.com/> (referer: None)
2021-11-30 16:35:18 [scrapy.spidermiddlewares.httperror] INFO: Ignoring response <404 https://www.digitalalloys.com/>: HTTP status code is not handled or not allowed


...

2021-11-30 16:45:52 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://myfavoriteartplace.com> (referer: None)
2021-11-30 16:45:52 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://superiorpackagingandfinishing.com/> (referer: None)


site_url https://www.itwgraphics.com/ site_url_close ['https://www.itwgraphics.com/', 'https://www.lrtgraphics.com/', 'https://www.isggraphics.com/']
site_url_close[0] https://www.itwgraphics.com/
ii 1061 self.companies[ii] ITW Graphics
out1 true iout1 853 url https://www.itwgraphics.com/
companies[0] Nuhill Technologies this url https://www.itwgraphics.com/
site_url https://www.kolibriartstudio.com site_url_close ['https://www.kolibriartstudio.com', 'https://www.artbeatstudios.com', 'http://www.barnstormstudio.com']
site_url_close[0] https://www.kolibriartstudio.com
ii 1060 self.companies[ii] Kolibri Art Studio
out1 false iout2 104 url https://www.kolibriartstudio.com
companies[0] Imaging Spectrum this url https://www.kolibriartstudio.com
site_url https://www.docunetworks.com/ site_url_close ['http://www.docunetworks.com', 'http://www.dotworks.com', 'http://www.lpbnetworks.com/']
site_url_close[0] http://www.docunetworks.com
ii 1054 self.companies[ii] Docunet
out1 true iout1 854 url h

2021-11-30 16:45:55 [scrapy.core.engine] DEBUG: Crawled (403) <GET https://www.scanningamerica.com> (referer: None)
2021-11-30 16:45:55 [scrapy.spidermiddlewares.httperror] INFO: Ignoring response <403 https://www.scanningamerica.com>: HTTP status code is not handled or not allowed


site_url https://superiorpackagingandfinishing.com/ site_url_close ['https://www.superiorpackagingandfinishing.com', 'https://shipritepackaging.com/', 'https://inovarpackaging.com/']
site_url_close[0] https://www.superiorpackagingandfinishing.com
ii 1050 self.companies[ii] Superior Packaging and Finishing
out1 true iout1 857 url https://superiorpackagingandfinishing.com/
companies[0] PRO-COPY, INC. this url https://superiorpackagingandfinishing.com/


2021-11-30 16:45:55 [scrapy.downloadermiddlewares.redirect] DEBUG: Redirecting (301) to <GET http://www.scngprinting.com> from <GET http://freedomprinting.net/>
2021-11-30 16:45:55 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.sdicgm.com/> (referer: None)
